In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

#2017, 2018, 2019 data for weather availible
#comp sci -  421 panels 
#rec - 660 panel
#fine arts - 424 panels

In [2]:
data_df =  pd.read_csv('UMass_PowerConsumption.csv', parse_dates=['Site Time'], )
weather_df = pd.read_csv('Amherst_Weather_Cleaned.csv',)

In [3]:
weather_df.head(10)

,Unnamed: 0,Year,Month,Day,DHI,DNI Max,DNI Min,DNI,GHI,Clearsky DHI,Clearsky DNI,Clearsky GHI,Wind Speed,Precipitable Water,Wind Direction,Relative Humidity,Temperature,Pressure
0,0,2017,1,1,82.625000,838,310,663.500000,286.250000,73.750,729.000000,304.625000,0.237500,0.551875,283.912500,89.031250,0.500000,1000.00
1,1,2017,1,2,109.625000,649,150,330.625000,210.250000,68.250,716.875000,297.125000,0.112500,1.161000,101.950000,83.982500,0.500000,1010.00
2,2,2017,1,4,125.500000,384,31,194.750000,175.250000,74.750,627.000000,285.000000,0.225000,1.462250,248.150000,100.000000,2.500000,980.00
3,3,2017,1,5,74.500000,769,493,652.750000,279.625000,74.625,736.625000,313.375000,0.312500,0.483875,264.037500,58.532500,-2.500000,990.00
4,4,2017,1,6,77.250000,845,521,664.125000,287.500000,73.250,747.750000,317.500000,0.250000,0.429875,309.150000,61.963750,-2.875000,1000.00
5,5,2017,1,7,104.000000,2,2,2.000000,105.000000,74.000,723.000000,286.000000,0.200000,0.686000,14.000000,69.400000,-7.000000,1010.00
6,6,2017,1,8,100.000000,770,299,539.375000,273.625000,96.750,676.000000,320.250000,0.275000,0.256125,296.637500,65.640000,-8.000000,1000.00
7,7,2017,1,9,90.375000,691,283,511.625000,261.375000,82.625,733.875000,325.375000,0.212500,0.305375,246.612500,63.127500,-8.875000,1020.00
8,8,2017,1,10,120.125000,474,107,312.000000,225.875000,91.375,651.875000,311.625000,0.325000,0.894750,171.287500,77.673750,-3.375000,1011.25
9,9,2017,1,11,76.222222,914,51,456.222222,233.333333,51.000,711.333333,286.333333,0.311111,0.522667,223.655556,92.948889,5.111111,1000.00


In [4]:
weather_df = weather_df.drop(['Unnamed: 0'], axis = 1)

In [5]:
weather_df.head()

,Year,Month,Day,DHI,DNI Max,DNI Min,DNI,GHI,Clearsky DHI,Clearsky DNI,Clearsky GHI,Wind Speed,Precipitable Water,Wind Direction,Relative Humidity,Temperature,Pressure
0,2017,1,1,82.625,838,310,663.500,286.250,73.750,729.000,304.625,0.2375,0.551875,283.9125,89.03125,0.500,1000.0
1,2017,1,2,109.625,649,150,330.625,210.250,68.250,716.875,297.125,0.1125,1.161000,101.9500,83.98250,0.500,1010.0
2,2017,1,4,125.500,384,31,194.750,175.250,74.750,627.000,285.000,0.2250,1.462250,248.1500,100.00000,2.500,980.0
3,2017,1,5,74.500,769,493,652.750,279.625,74.625,736.625,313.375,0.3125,0.483875,264.0375,58.53250,-2.500,990.0
4,2017,1,6,77.250,845,521,664.125,287.500,73.250,747.750,317.500,0.2500,0.429875,309.1500,61.96375,-2.875,1000.0


In [6]:
data_df.head(70)

,Site Time,UMASS Amherst - Robsham Visitors Center null,UMASS Amherst - Lot 44 Carport null,UMASS Amherst - Champions Center null,UMASS Amherst - Computer Science null,UMASS Amherst - Fine Arts Center null,UMASS Amherst - Recreation Center null,UMASS Amherst - University Police null,UMASS Amherst - Lot 25 Carport null
0,2017-01-01,80.5,0,0,0,0.00,0,0.000,0
1,2017-01-02,59.3,0,0,0,0.00,0,0.000,0
2,2017-01-03,22.8,34.629,0,0,0.00,0,0.000,0
3,2017-01-04,76.6,0,0,0,0.00,0,0.000,0
4,2017-01-05,224,0,0,0,0.00,0,0.000,0
...,...,...,...,...,...,...,...,...,...
65,2017-03-07,67.1,"1,560.3",153.12,100.33,69.39,154.38,18.381,"1,238.7"
66,2017-03-08,399.9,"8,702",919.67,589.84,564.75,944,118.687,"6,632.9"
67,2017-03-09,536.6,"11,252.6","1,269.63",822.65,879.26,"1,309.44",156.511,"8,924.1"
68,2017-03-10,103.6,"2,467.2",247.6,150.63,164.12,242.1,23.675,"2,226.9"


In [7]:
data_df['Day'] = [d.day for d in data_df['Site Time']]
data_df['Month'] = [d.month for d in data_df['Site Time']]
data_df['Year'] = [d.year for d in data_df['Site Time']]
data_df['Date'] = [d.date() for d in data_df['Site Time']]

In [8]:
data_df.head(10)

,Site Time,UMASS Amherst - Robsham Visitors Center null,UMASS Amherst - Lot 44 Carport null,UMASS Amherst - Champions Center null,UMASS Amherst - Computer Science null,UMASS Amherst - Fine Arts Center null,UMASS Amherst - Recreation Center null,UMASS Amherst - University Police null,UMASS Amherst - Lot 25 Carport null,Day,Month,Year,Date
0,2017-01-01,80.5,0,0,0,0.0,0,0.0,0,1,1,2017,2017-01-01
1,2017-01-02,59.3,0,0,0,0.0,0,0.0,0,2,1,2017,2017-01-02
2,2017-01-03,22.8,34.629,0,0,0.0,0,0.0,0,3,1,2017,2017-01-03
3,2017-01-04,76.6,0,0,0,0.0,0,0.0,0,4,1,2017,2017-01-04
4,2017-01-05,224,0,0,0,0.0,0,0.0,0,5,1,2017,2017-01-05
5,2017-01-06,194.9,0,0,0,0.0,0,0.0,0,6,1,2017,2017-01-06
6,2017-01-07,22.8,53.598,0,0,0.0,0,0.0,0,7,1,2017,2017-01-07
7,2017-01-08,7.2,110.447,0,0,0.0,0,0.0,0,8,1,2017,2017-01-08
8,2017-01-09,9.9,101.987,0,0,0.0,0,0.0,0,9,1,2017,2017-01-09
9,2017-01-10,9.7,96.476,0,0,0.0,0,0.0,0,10,1,2017,2017-01-10


In [9]:
data_new_df = data_df[data_df.Year != 2020].drop(['UMASS Amherst - Champions Center null','UMASS Amherst - Robsham Visitors Center null','UMASS Amherst - Lot 44 Carport null', 'UMASS Amherst - University Police null', 'UMASS Amherst - Lot 25 Carport null' ], axis=1)

In [10]:
data_new_df.head(10)

,Site Time,UMASS Amherst - Computer Science null,UMASS Amherst - Fine Arts Center null,UMASS Amherst - Recreation Center null,Day,Month,Year,Date
0,2017-01-01,0,0.0,0,1,1,2017,2017-01-01
1,2017-01-02,0,0.0,0,2,1,2017,2017-01-02
2,2017-01-03,0,0.0,0,3,1,2017,2017-01-03
3,2017-01-04,0,0.0,0,4,1,2017,2017-01-04
4,2017-01-05,0,0.0,0,5,1,2017,2017-01-05
5,2017-01-06,0,0.0,0,6,1,2017,2017-01-06
6,2017-01-07,0,0.0,0,7,1,2017,2017-01-07
7,2017-01-08,0,0.0,0,8,1,2017,2017-01-08
8,2017-01-09,0,0.0,0,9,1,2017,2017-01-09
9,2017-01-10,0,0.0,0,10,1,2017,2017-01-10


In [11]:
data_new_df['UMASS Amherst - Computer Science null'] = data_new_df['UMASS Amherst - Computer Science null'].str.replace(',','').astype(float)
#data_new_df['UMASS Amherst - Fine Arts Center null'] = data_new_df['UMASS Amherst - Fine Arts Center null'].str.replace(',','').astype(float)
data_new_df['UMASS Amherst - Recreation Center null'] = data_new_df['UMASS Amherst - Recreation Center null'].str.replace(',','').astype(float)


In [12]:
data_new_df.head(10)

,Site Time,UMASS Amherst - Computer Science null,UMASS Amherst - Fine Arts Center null,UMASS Amherst - Recreation Center null,Day,Month,Year,Date
0,2017-01-01,0.0,0.0,0.0,1,1,2017,2017-01-01
1,2017-01-02,0.0,0.0,0.0,2,1,2017,2017-01-02
2,2017-01-03,0.0,0.0,0.0,3,1,2017,2017-01-03
3,2017-01-04,0.0,0.0,0.0,4,1,2017,2017-01-04
4,2017-01-05,0.0,0.0,0.0,5,1,2017,2017-01-05
5,2017-01-06,0.0,0.0,0.0,6,1,2017,2017-01-06
6,2017-01-07,0.0,0.0,0.0,7,1,2017,2017-01-07
7,2017-01-08,0.0,0.0,0.0,8,1,2017,2017-01-08
8,2017-01-09,0.0,0.0,0.0,9,1,2017,2017-01-09
9,2017-01-10,0.0,0.0,0.0,10,1,2017,2017-01-10


In [13]:
data_new_df['UMASS Amherst - Computer Science null'] = data_new_df['UMASS Amherst - Computer Science null'] / 421
data_new_df['UMASS Amherst - Fine Arts Center null'] = data_new_df['UMASS Amherst - Fine Arts Center null'] / 424
data_new_df['UMASS Amherst - Recreation Center null'] = data_new_df['UMASS Amherst - Recreation Center null'] / 660

In [14]:
data_new_df.head(70)

,Site Time,UMASS Amherst - Computer Science null,UMASS Amherst - Fine Arts Center null,UMASS Amherst - Recreation Center null,Day,Month,Year,Date
0,2017-01-01,0.000000,0.000000,0.000000,1,1,2017,2017-01-01
1,2017-01-02,0.000000,0.000000,0.000000,2,1,2017,2017-01-02
2,2017-01-03,0.000000,0.000000,0.000000,3,1,2017,2017-01-03
3,2017-01-04,0.000000,0.000000,0.000000,4,1,2017,2017-01-04
4,2017-01-05,0.000000,0.000000,0.000000,5,1,2017,2017-01-05
...,...,...,...,...,...,...,...,...
65,2017-03-07,0.238314,0.163656,0.233909,7,3,2017,2017-03-07
66,2017-03-08,1.401045,1.331958,1.430303,8,3,2017,2017-03-08
67,2017-03-09,1.954038,2.073726,1.984000,9,3,2017,2017-03-09
68,2017-03-10,0.357791,0.387075,0.366818,10,3,2017,2017-03-10


In [15]:
data_new_df = data_new_df.drop(['Site Time'], axis=1)
data_new_df.head(10)

,UMASS Amherst - Computer Science null,UMASS Amherst - Fine Arts Center null,UMASS Amherst - Recreation Center null,Day,Month,Year,Date
0,0.0,0.0,0.0,1,1,2017,2017-01-01
1,0.0,0.0,0.0,2,1,2017,2017-01-02
2,0.0,0.0,0.0,3,1,2017,2017-01-03
3,0.0,0.0,0.0,4,1,2017,2017-01-04
4,0.0,0.0,0.0,5,1,2017,2017-01-05
5,0.0,0.0,0.0,6,1,2017,2017-01-06
6,0.0,0.0,0.0,7,1,2017,2017-01-07
7,0.0,0.0,0.0,8,1,2017,2017-01-08
8,0.0,0.0,0.0,9,1,2017,2017-01-09
9,0.0,0.0,0.0,10,1,2017,2017-01-10


In [16]:
weather_df.head(10)

,Year,Month,Day,DHI,DNI Max,DNI Min,DNI,GHI,Clearsky DHI,Clearsky DNI,Clearsky GHI,Wind Speed,Precipitable Water,Wind Direction,Relative Humidity,Temperature,Pressure
0,2017,1,1,82.625000,838,310,663.500000,286.250000,73.750,729.000000,304.625000,0.237500,0.551875,283.912500,89.031250,0.500000,1000.00
1,2017,1,2,109.625000,649,150,330.625000,210.250000,68.250,716.875000,297.125000,0.112500,1.161000,101.950000,83.982500,0.500000,1010.00
2,2017,1,4,125.500000,384,31,194.750000,175.250000,74.750,627.000000,285.000000,0.225000,1.462250,248.150000,100.000000,2.500000,980.00
3,2017,1,5,74.500000,769,493,652.750000,279.625000,74.625,736.625000,313.375000,0.312500,0.483875,264.037500,58.532500,-2.500000,990.00
4,2017,1,6,77.250000,845,521,664.125000,287.500000,73.250,747.750000,317.500000,0.250000,0.429875,309.150000,61.963750,-2.875000,1000.00
5,2017,1,7,104.000000,2,2,2.000000,105.000000,74.000,723.000000,286.000000,0.200000,0.686000,14.000000,69.400000,-7.000000,1010.00
6,2017,1,8,100.000000,770,299,539.375000,273.625000,96.750,676.000000,320.250000,0.275000,0.256125,296.637500,65.640000,-8.000000,1000.00
7,2017,1,9,90.375000,691,283,511.625000,261.375000,82.625,733.875000,325.375000,0.212500,0.305375,246.612500,63.127500,-8.875000,1020.00
8,2017,1,10,120.125000,474,107,312.000000,225.875000,91.375,651.875000,311.625000,0.325000,0.894750,171.287500,77.673750,-3.375000,1011.25
9,2017,1,11,76.222222,914,51,456.222222,233.333333,51.000,711.333333,286.333333,0.311111,0.522667,223.655556,92.948889,5.111111,1000.00


In [17]:
data_new_df = data_new_df[(data_new_df != 0).all(1)]

In [18]:
final_df = weather_df.merge(data_new_df, left_on = ('Month', 'Day', 'Year'),right_on = ('Month', 'Day', 'Year'), how = 'inner') 

In [19]:
#final_df = pd.concat([data_new_df, weather_df], axis=1, ignore_index=True)
#final_df.columns = ('UMASS Amherst - Computer Science null', 'UMASS Amherst - Fine Arts Center null', 'UMASS Amherst - Recreation Center null', 'Day', 'Month', 'Year', 'Date', 'DHI', 'DNI Max', 'DNI Min', 'DNI', 'GHI', 'Clearsky DHI', 'Clearsky DNI', 'Clearsky GHI', 'Wind Speed','Precipitable Water', 'Wind Direction', 'Relative Humidity', 'Temperature', 'Pressure')

In [20]:
final_df.head(10)

,Year,Month,Day,DHI,DNI Max,DNI Min,DNI,GHI,Clearsky DHI,Clearsky DNI,...,Wind Speed,Precipitable Water,Wind Direction,Relative Humidity,Temperature,Pressure,UMASS Amherst - Computer Science null,UMASS Amherst - Fine Arts Center null,UMASS Amherst - Recreation Center null,Date
0,2017,2,3,76.100000,808,194,606.200000,305.800000,86.300000,697.200000,...,0.480000,0.383400,277.570000,67.023000,-3.500000,1000.000000,1.061948,1.411040,1.095000,2017-02-03
1,2017,2,4,88.100000,874,286,676.200000,345.400000,88.200000,694.800000,...,0.600000,0.363900,284.940000,67.680000,-3.900000,1000.000000,1.409026,1.852061,1.453818,2017-02-04
2,2017,2,5,122.888889,693,31,356.222222,278.333333,117.666667,615.888889,...,0.566667,0.886000,209.822222,76.916667,-1.000000,992.222222,0.810475,0.826453,0.818667,2017-02-05
3,2017,2,6,77.666667,927,4,649.222222,336.444444,72.222222,752.111111,...,0.611111,0.731889,294.100000,61.147778,-1.777778,1000.000000,1.234228,1.570186,1.250591,2017-02-06
4,2017,2,8,109.800000,850,33,466.400000,295.700000,96.600000,657.000000,...,0.500000,1.026600,284.440000,98.111000,2.800000,985.000000,0.657363,1.043031,0.712076,2017-02-08
5,2017,2,10,97.700000,969,213,648.100000,347.400000,75.300000,799.600000,...,0.800000,0.313100,303.320000,79.364000,-9.400000,1000.000000,0.630855,0.915967,0.251333,2017-02-10
6,2017,2,11,231.333333,71,44,60.000000,263.666667,174.333333,717.000000,...,0.100000,1.254333,93.700000,93.980000,-2.000000,990.000000,0.013872,0.314078,0.006076,2017-02-11
7,2017,2,13,151.100000,634,54,364.500000,288.500000,134.700000,593.500000,...,1.310000,0.568900,313.540000,91.813000,-4.100000,984.000000,0.021354,1.335474,0.008955,2017-02-13
8,2017,2,14,88.300000,907,269,715.200000,395.300000,86.300000,756.600000,...,0.280000,0.607600,262.120000,88.257000,-3.300000,990.000000,0.177933,1.911908,0.117985,2017-02-14
9,2017,2,15,173.875000,457,7,231.000000,281.375000,151.500000,602.375000,...,0.237500,1.008250,145.400000,99.466250,0.750000,975.000000,0.253753,0.659790,0.230424,2017-02-15


In [21]:
#final_df = final_df[final_df['UMASS Amherst - Fine Arts Center null'] != 0]

In [22]:
final_df.head(40)

,Year,Month,Day,DHI,DNI Max,DNI Min,DNI,GHI,Clearsky DHI,Clearsky DNI,...,Wind Speed,Precipitable Water,Wind Direction,Relative Humidity,Temperature,Pressure,UMASS Amherst - Computer Science null,UMASS Amherst - Fine Arts Center null,UMASS Amherst - Recreation Center null,Date
0,2017,2,3,76.100000,808,194,606.200000,305.800000,86.300000,697.200000,...,0.480000,0.383400,277.570000,67.023000,-3.500000,1000.000000,1.061948,1.411040,1.095000,2017-02-03
1,2017,2,4,88.100000,874,286,676.200000,345.400000,88.200000,694.800000,...,0.600000,0.363900,284.940000,67.680000,-3.900000,1000.000000,1.409026,1.852061,1.453818,2017-02-04
2,2017,2,5,122.888889,693,31,356.222222,278.333333,117.666667,615.888889,...,0.566667,0.886000,209.822222,76.916667,-1.000000,992.222222,0.810475,0.826453,0.818667,2017-02-05
3,2017,2,6,77.666667,927,4,649.222222,336.444444,72.222222,752.111111,...,0.611111,0.731889,294.100000,61.147778,-1.777778,1000.000000,1.234228,1.570186,1.250591,2017-02-06
4,2017,2,8,109.800000,850,33,466.400000,295.700000,96.600000,657.000000,...,0.500000,1.026600,284.440000,98.111000,2.800000,985.000000,0.657363,1.043031,0.712076,2017-02-08
5,2017,2,10,97.700000,969,213,648.100000,347.400000,75.300000,799.600000,...,0.800000,0.313100,303.320000,79.364000,-9.400000,1000.000000,0.630855,0.915967,0.251333,2017-02-10
6,2017,2,11,231.333333,71,44,60.000000,263.666667,174.333333,717.000000,...,0.100000,1.254333,93.700000,93.980000,-2.000000,990.000000,0.013872,0.314078,0.006076,2017-02-11
7,2017,2,13,151.100000,634,54,364.500000,288.500000,134.700000,593.500000,...,1.310000,0.568900,313.540000,91.813000,-4.100000,984.000000,0.021354,1.335474,0.008955,2017-02-13
8,2017,2,14,88.300000,907,269,715.200000,395.300000,86.300000,756.600000,...,0.280000,0.607600,262.120000,88.257000,-3.300000,990.000000,0.177933,1.911908,0.117985,2017-02-14
9,2017,2,15,173.875000,457,7,231.000000,281.375000,151.500000,602.375000,...,0.237500,1.008250,145.400000,99.466250,0.750000,975.000000,0.253753,0.659790,0.230424,2017-02-15


In [23]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 946 entries, 0 to 945
Data columns (total 21 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Year                                    946 non-null    int64  
 1   Month                                   946 non-null    int64  
 2   Day                                     946 non-null    int64  
 3   DHI                                     946 non-null    float64
 4   DNI Max                                 946 non-null    int64  
 5   DNI Min                                 946 non-null    int64  
 6   DNI                                     946 non-null    float64
 7   GHI                                     946 non-null    float64
 8   Clearsky DHI                            946 non-null    float64
 9   Clearsky DNI                            946 non-null    float64
 10  Clearsky GHI                            946 non-null    float6

In [24]:
print(final_df)

     Year  Month  Day         DHI  DNI Max  DNI Min         DNI         GHI  \
0    2017      2    3   76.100000      808      194  606.200000  305.800000   
1    2017      2    4   88.100000      874      286  676.200000  345.400000   
2    2017      2    5  122.888889      693       31  356.222222  278.333333   
3    2017      2    6   77.666667      927        4  649.222222  336.444444   
4    2017      2    8  109.800000      850       33  466.400000  295.700000   
..    ...    ...  ...         ...      ...      ...         ...         ...   
941  2019     12   27   87.750000      290        3   84.500000  109.125000   
942  2019     12   28   49.800000      891       31  619.100000  239.100000   
943  2019     12   29   68.300000      631       23  365.500000  187.000000   
944  2019     12   30   86.800000      217       12   54.400000  101.600000   
945  2019     12   31   92.900000      407       21  181.900000  147.400000   

     Clearsky DHI  Clearsky DNI  ...  Wind Speed  P

In [25]:
final_df.to_csv('Umass_total_data.csv')

In [26]:
#'UMASS Amherst - Fine Arts Center null','UMASS Amherst - Recreation Center null'